In [46]:
from Core_Game_Parts import *
import pygame
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
import time
import os
os.environ["SDL_VIDEODRIVER"] = "windows" 

In [47]:
def model():
    """Builds a simple feedforward neural network model.

    Returns:
        model: A Keras Sequential model instance.
    """
    model=Sequential(
        [Dense(32,activation='relu',input_shape=(4,)),
         Dense(16,activation='relu'),
         Dense(3,activation='linear')]
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)
    model.compile(optimizer=optimizer, loss='mse')
    return model
ai_model=model()
ai_model.summary()
    

Model: "sequential_41"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_123 (Dense)               │ (None, 32)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_124 (Dense)               │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_125 (Dense)               │ (None, 3)              │            51 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 739 (2.89 KB)

 Trainable params: 739 (2.89 KB)

 Non-trainable params: 0 (0.00 B)

In [48]:
def run_trained_agent(weights_path):
    """
    Loads a trained agent and runs the visual simulation with corrected logic.
    """
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Trained AI Agent")
    font = pygame.font.SysFont(None, 36)
    crash_font = pygame.font.SysFont(None, 72)
    track_surface = pygame.image.load(TRACK_IMAGE_PATH).convert()

    ai_model = model()
    try:
        ai_model.load_weights(weights_path)
        print(f"Successfully loaded weights from: {weights_path}")
    except Exception as e:
        print(f"Error loading weights: {e}")
        pygame.quit()
        return
    action_counts = {0: 0, 1: 0, 2: 0}  # To track the number of each action taken
    car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # 1. Get state (Corrected to use 4 inputs)
        distances, ray_endpoints = ray_casting(car, track_surface)
        normalized_speed = car.speed / MAX_SPEED
        state = np.array(distances + [normalized_speed])
        state = np.reshape(state, [1, 4])
        # 2. Predict the best action
        q_values = ai_model.predict(state, verbose=0)
        action = np.argmax(q_values[0])
        action_map = {0: "Left", 1: "Right", 2: "Brake"}
        action_counts[action] += 1
        # 3. Execute action (using the same physics as the training function)
        car.speed += ACCELERATION
        if car.speed > 0:
            speed_factor = car.speed / MAX_SPEED
            dynamic_turn_angle = MAX_TURN_ANGLE - (speed_factor) * (MAX_TURN_ANGLE - MIN_TURN_ANGLE)
            if action == 0: car.angle += dynamic_turn_angle
            elif action == 1: car.angle -= dynamic_turn_angle
        if action == 2: car.speed -= BRAKE_FORCE
        
        car.speed -= FRICTION
        car.speed = max(0, min(car.speed, MAX_SPEED))
        car.move()

        # 4. Check for crash (Corrected to use BG_COLOR)
        crashed = False
        try:
            pixel_color = track_surface.get_at((int(car.x), int(car.y)))[:3]
            if pixel_color == DRAW_COLOR:
                crashed = True
        except IndexError:
            crashed = True
        
        if crashed:
            crash_text = crash_font.render("CRASHED! Resetting...", True, (255, 0, 0))
            text_rect = crash_text.get_rect(center=(SCREEN_WIDTH/2, SCREEN_HEIGHT/2))
            screen.blit(crash_text, text_rect)
            pygame.display.update()
            pygame.time.wait(2000)
            car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
            print("Total actions taken by the agent during the simulation:")
            for action, count in action_counts.items():
                print(f"Action {action}: {count}")
            action_counts = {0: 0, 1: 0, 2: 0}
            
        # 5. Render everything
        screen.blit(track_surface, (0, 0))
        car.draw(screen)
        for point in ray_endpoints:
            pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), point, 1)
        
        info_text = f"Action: {action_map[action]} | Speed: {car.speed:.2f}"
        text_surface = font.render(info_text, True, (255, 255, 255))
        screen.blit(text_surface, (20, 20))

        pygame.display.update()
        clock.tick(60)

    pygame.quit()   

# --- RUN THE SIMULATION WITH YOUR TRAINED WEIGHTS ---
# ------------------------------------------------------------------
# IMPORTANT: Change this to the name of your saved weights file!
WEIGHTS_FILENAME = "finetuned_weights_episode_600.weights.h5"
# ------------------------------------------------------------------

# This checks if a pygame window is already open to avoid errors
if not pygame.display.get_init():
    run_trained_agent(WEIGHTS_FILENAME)
else:
    # If you get this message, restart the notebook kernel before running this cell again.
    print("Pygame window may already be open. Please restart the kernel to run again.")

Error loading weights: A total of 3 objects could not be loaded. Example error message for object <Dense name=dense_126, built=True>:

The shape of the target variable and the shape of the target value in `variable.assign(value)` must match. variable.shape=(4, 32), Received: value.shape=(4, 64). Target variable: <Variable path=sequential_42/dense_126/kernel, shape=(4, 32), dtype=float32, value=[[-0.23685893 -0.06073299  0.33682036 -0.08710808  0.3521924  -0.04636797
   0.2709021   0.35026324 -0.07572749 -0.02431053  0.40008897  0.22819954
  -0.15395638  0.36628926  0.36562598 -0.14341685  0.15896487 -0.054066
   0.23976481 -0.35768837 -0.25903717 -0.049725    0.02800611 -0.26188195
   0.15255195  0.40119267 -0.11390129  0.2985571   0.0814296   0.02732408
   0.2983405  -0.13282526]
 [-0.28503448 -0.25244987 -0.09811771 -0.11161521  0.0056546  -0.27458578
  -0.06426612 -0.21189958 -0.33281574  0.35008764  0.12161958 -0.3475892
   0.08241609 -0.37503356  0.23186898  0.12444264 -0.25257796

c:\Users\Yash\OneDrive\Desktop\Neural Networks Car Project\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:797: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [57]:
def run_final_agent(weights_path):
    """
    Loads the imitation-trained agent and runs the visual simulation correctly.
    """
    os.environ["SDL_VIDEODRIVER"] = "windows"
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    clock = pygame.time.Clock()
    pygame.display.set_caption("Final Trained Agent")
    font = pygame.font.SysFont(None, 36)
    track_surface = pygame.image.load(TRACK_IMAGE_PATH).convert()

    # Define the model architecture EXACTLY as it was for training in Phase 2
    def model_for_testing():
        net = Sequential([
            Dense(64, activation='relu', input_shape=(4,)),
            Dense(64, activation='relu'),
            Dense(3, activation='linear') # Use linear for argmax, it's more stable
        ])
        net.compile() # Compile is not strictly needed for prediction but is good practice
        return net

    ai_model = model_for_testing()
    try:
        ai_model.load_weights(weights_path)
        print(f"Successfully loaded final weights from: {weights_path}")
    except Exception as e:
        print(f"Error loading weights: {e}")
        pygame.quit(); return
    action_counts = {0: 0, 1: 0, 2: 0}
    # This assumes your Car class and ray_casting function are defined in a cell above
    car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
    running = True
    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                running = False

        # 1. Get state
        distances, ray_endpoints = ray_casting(car, track_surface)
        state = np.array(distances + [car.speed / MAX_SPEED])
        state = np.reshape(state, [1, 4])
        
        # 2. Predict best action
        prediction = ai_model.predict(state, verbose=0)
        action = np.argmax(prediction[0])
        action_map = {0: "Left", 1: "Right", 2: "Brake"}
        action_counts[action] += 1
        
        # 3. Execute action
        if action != 2:
            car.speed += ACCELERATION
        if car.speed > 0:
            speed_factor = car.speed / MAX_SPEED
            turn = MAX_TURN_ANGLE - (speed_factor) * (MAX_TURN_ANGLE - MIN_TURN_ANGLE)
            if action == 0: car.angle += turn
            elif action == 1: car.angle -= turn
        if action == 2: car.speed -= BRAKE_FORCE
        car.speed -= FRICTION
        car.speed = max(0, min(car.speed, MAX_SPEED))
        car.move()
        
        # 4. Check for crash (CORRECTED LOGIC)
        crashed = False
        try:
            # Check if the car is on the WALL (BG_COLOR)
            if track_surface.get_at((int(car.x), int(car.y)))[:3] == DRAW_COLOR:
                crashed = True
        except IndexError:
            crashed = True # Crashed if out of screen bounds
        
        if crashed:
            # Simple reset on crash
            car = Car(CAR_IMAGE_PATH, DEFAULT_START_X, DEFAULT_START_Y, angle=DEFAULT_START_ANGLE)
            print("Car crashes!")
            print("Total actions taken by the agent during the simulation:")
            for action, count in action_counts.items():
                print(f"Action {action}: {count}")
            action_counts = {0: 0, 1: 0, 2: 0}

        # 5. Render everything
        screen.blit(track_surface, (0, 0))
        car.draw(screen)
        for point in ray_endpoints:
            pygame.draw.line(screen, (0, 255, 0), (car.x, car.y), point, 1)
        
        info_text = f"Action: {action_map[action]} | Speed: {car.speed:.2f}"
        text_surface = font.render(info_text, True, (255, 255, 255))
        screen.blit(text_surface, (20, 20))

        pygame.display.update()
        clock.tick(60)
        
    pygame.quit()

# --- RUN YOUR FINAL AGENT ---
# ------------------------------------------------------------------
# IMPORTANT: Change this to the name of the weights file saved in Phase 2
WEIGHTS_FILENAME = "final_imitation.weights.h5"
# ------------------------------------------------------------------

if not pygame.display.get_init():
    run_final_agent(WEIGHTS_FILENAME)
else:
    print("Pygame window may already be open. Please restart the kernel to run again.")

Successfully loaded final weights from: final_imitation.weights.h5
Car crashes!
Total actions taken by the agent during the simulation:
Action 0: 1
Action 1: 49
Action 2: 0
